In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

os.chdir(r'C:\Users\emmalee.dolfi\Desktop\springboard_datascience-master\Capstone\data')

In [2]:
parksInfo = pd.read_excel('NYC_parksInfo_all.xlsx', index_col = 'index')
#parksInfo.head()

parksInfo_dummied = pd.get_dummies(parksInfo.drop(['GISPROPNUM','TPL_P_NAME','SIGNNAME', 'OBJECTID','FREQUENCY', 'SUM_SVCAREA'], axis=1))
parksInfo_dummied.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 0 to 97
Data columns (total 50 columns):
ACRES                                     98 non-null float64
handball                                  98 non-null int64
tennis                                    98 non-null int64
basketball                                98 non-null int64
adult_soft                                98 non-null int64
track_and                                 98 non-null int64
play_area_count                           98 non-null int64
preserve_count                            98 non-null int64
spray_count                               98 non-null int64
SUM_TOTPOPSVCA                            98 non-null int64
SUM_KIDSVCA                               98 non-null int64
SUM_YOUNGPROSVCA                          98 non-null int64
SUM_SENIORSVCA                            98 non-null int64
SUM_HHILOWSVCA                            98 non-null int64
SUM_HHIMEDSVCA                            98 non-null i

In [4]:
parksInfo_dummied.tweet_count.describe()

def label_tweets (row):
   if row['tweet_count'] <= 50:
    return 0
   if row['tweet_count'] > 50 and row['tweet_count'] <= 200:
    return 1
   else:
    return 2

parksInfo_dummied['tweet_class'] = parksInfo_dummied.apply(lambda row: label_tweets (row),axis=1)
parksInfo_dummied.head()

,ACRES,handball,tennis,basketball,adult_soft,track_and,play_area_count,preserve_count,spray_count,SUM_TOTPOPSVCA,...,TYPECATEGO_Playground,TYPECATEGO_Recreation Field/Courts,TYPECATEGO_Restrictive Declaration,TYPECATEGO_Schoolyard-Playground,TYPECATEGO_Schoolyards to Playgrounds,TYPECATEGO_Tracking,TYPECATEGO_Triangle/Plaza,TYPECATEGO_Undeveloped,TYPECATEGO_Waterfront Facility,tweet_class
index,,,,,,,,,,,,,,,,,,,,,
0,1.874,0,0,3,1,0,4,0,1,26177,...,0,0,0,0,0,0,0,0,0,2
1,6.379,4,0,1,0,0,5,0,1,29767,...,0,0,0,0,0,0,0,0,0,1
2,35.713,12,8,4,4,0,7,0,2,42017,...,0,0,0,0,0,0,0,0,0,2
3,0.181,0,0,0,0,0,0,0,0,28730,...,0,0,0,0,0,0,0,0,0,1
4,0.293,0,0,0,0,0,1,0,1,32449,...,1,0,0,0,0,0,0,0,0,0


In [5]:
parksInfo_sub = parksInfo_dummied[['ACRES','SUM_HHILOWSVCA','handball']]

y = np.array(parksInfo.tweet_count)
X = parksInfo_sub

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
print('Training Features Shape:', train_X.shape)
print('Training Labels Shape:', train_y.shape)
print('Testing Features Shape:', test_X.shape)
print('Testing Labels Shape:', test_y.shape)

('Training Features Shape:', (78, 3))
('Training Labels Shape:', (78,))
('Testing Features Shape:', (20, 3))
('Testing Labels Shape:', (20,))


In [7]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000)
# Train the model on training data
rf.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [8]:
predict_test = rf.predict(test_X)

In [9]:
print "Train Accuracy :: ", rf.score(train_X, train_y)
print "Test Accuracy  :: ", rf.score(test_X, test_y)

Train Accuracy ::  0.8435007431352982
Test Accuracy  ::  0.11441341598166499


In [10]:
y = np.array(parksInfo_dummied.tweet_class)
X = parksInfo_dummied.drop(['tweet_class','tweet_count'], axis=1)

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [11]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
# Train the model on training data
rf.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [12]:
predict_test = rf.predict(test_X)

In [13]:
print "Train Accuracy :: ", accuracy_score(train_y, rf.predict(train_X))
print "Test Accuracy  :: ", accuracy_score(test_y, predict_test)

Train Accuracy ::  0.9615384615384616
Test Accuracy  ::  0.6


In [14]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {'n_estimators': range(1,50)}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid)

grid_search.fit(train_X, train_y)
grid_search.best_params_

{'n_estimators': 19}

In [38]:
rf = RandomForestClassifier(n_estimators=19)
# Train the model on training data
rf.fit(train_X, train_y)

predict_test = rf.predict(test_X)

print "Train Accuracy :: ", accuracy_score(train_y, rf.predict(train_X))
print "Test Accuracy  :: ", accuracy_score(test_y, predict_test)

Train Accuracy ::  0.9871794871794872
Test Accuracy  ::  0.6


In [32]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {'n_estimators': [100, 200, 300, 1000, 5000]}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid)

grid_search.fit(train_X, train_y)
grid_search.best_params_

{'n_estimators': 5000}

In [20]:
rf = RandomForestRegressor(n_estimators = 100)
# Train the model on training data
rf.fit(train_X, train_y)
predict_test = rf.predict(test_X)

print "Train Accuracy :: ", rf.score(train_X, train_y)
print "Test Accuracy  :: ", rf.score(test_X, test_y)

Train Accuracy ::  0.9035488909426987
Test Accuracy  ::  0.27654295532646045


In [85]:
from sklearn import svm

clf = svm.SVR()
clf.fit(train_X, train_y) 

clf.predict(test_X)

-0.04667604707362538

In [18]:
parksInfo_noCate = parksInfo_dummied.iloc[:,:27].drop(['total_amenities'], axis=1)
parksInfo_noCate['tweet_class'] = parksInfo_noCate.apply(lambda row: label_tweets (row),axis=1)
parksInfo_noCate.head()

,ACRES,handball,tennis,basketball,adult_soft,track_and,play_area_count,preserve_count,spray_count,SUM_TOTPOPSVCA,...,SUM_WHITE_SVC,SUM_BLACK_SVC,SUM_AMERINDSVC,SUM_ASIAN_SVC,SUM_PACIFICSVC,SUM_OTHRACESVC,SUM_RACE2UPSVC,SUM_HISP_SVC,tweet_count,tweet_class
index,,,,,,,,,,,,,,,,,,,,,
0,1.874,0,0,3,1,0,4,0,1,26177,...,21238,1136,51,1769,10,865,1104,3279,872,2
1,6.379,4,0,1,0,0,5,0,1,29767,...,23922,174,27,4523,11,758,349,1652,59,1
2,35.713,12,8,4,4,0,7,0,2,42017,...,34676,564,112,3412,20,1926,1320,4974,7554,2
3,0.181,0,0,0,0,0,0,0,0,28730,...,19543,1013,170,2651,13,4040,1298,8766,81,1
4,0.293,0,0,0,0,0,1,0,1,32449,...,6323,22999,125,548,10,1312,1130,3770,23,0


In [21]:
y = np.array(parksInfo_noCate.tweet_class)
X = parksInfo_noCate.drop(['tweet_class','tweet_count'], axis=1)

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Create the parameter grid based on the results of random search 
param_grid = {'n_estimators': range(1,50)}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid)

grid_search.fit(train_X, train_y)
grid_search.best_params_

{'n_estimators': 11}

In [22]:
rf = RandomForestClassifier(n_estimators=11)
# Train the model on training data
rf.fit(train_X, train_y)

predict_test = rf.predict(test_X)

print "Train Accuracy :: ", accuracy_score(train_y, rf.predict(train_X))
print "Test Accuracy  :: ", accuracy_score(test_y, predict_test)

Train Accuracy ::  0.9871794871794872
Test Accuracy  ::  0.65


In [44]:
importances = list(rf.feature_importances_)

feature_list = list(X.columns)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
print(sorted(feature_importances, key = lambda x: x[1], reverse=True))

[(u'ACRES', 0.14), (u'SUM_HHIHIGHSVCA', 0.08), (u'SUM_SENIORSVCA', 0.07), (u'SUM_HHIMEDSVCA', 0.07), (u'SUM_WHITE_SVC', 0.07), (u'SUM_KIDSVCA', 0.05), (u'SUM_TOTHHSVCA', 0.05), (u'SUM_AMERINDSVC', 0.05), (u'SUM_ASIAN_SVC', 0.05), (u'SUM_RACE2UPSVC', 0.05), (u'SUM_TOTPOPSVCA', 0.04), (u'SUM_YOUNGPROSVCA', 0.04), (u'SUM_HHILOWSVCA', 0.04), (u'SUM_BLACK_SVC', 0.04), (u'play_area_count', 0.03), (u'SUM_PACIFICSVC', 0.03), (u'SUM_HISP_SVC', 0.03), (u'handball', 0.02), (u'basketball', 0.02), (u'spray_count', 0.02), (u'SUM_OTHRACESVC', 0.02), (u'tennis', 0.0), (u'adult_soft', 0.0), (u'track_and', 0.0), (u'preserve_count', 0.0)]
